In [203]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

In [204]:
path = '../Datasets/rawData/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
print('Number of rows and columns in the train data set:',train.shape)
print('Number of rows and columns in the test data set:',test.shape)

Number of rows and columns in the train data set: (159571, 8)
Number of rows and columns in the test data set: (153164, 2)


In [205]:
# fig,ax = plt.subplots(2,3,figsize=(16,10))
# ax1,ax2,ax3,ax4,ax5,ax6 = ax.flatten()
# sns.countplot(train['toxic'],palette= 'RdBu',ax=ax1)
# sns.countplot(train['severe_toxic'], palette= 'viridis',ax=ax2)
# sns.countplot(train['obscene'], palette= 'Set1',ax=ax3)
# sns.countplot(train['threat'], palette= 'viridis',ax = ax4)
# sns.countplot(train['insult'], palette = 'magma',ax=ax5)
# sns.countplot(train['identity_hate'], palette = 'Set1', ax = ax6)

In [206]:
from sklearn.utils import resample

# Separate majority and minority classes
dataset_majority = train[train.toxic == 0]
dataset_minority = train[train.toxic == 1]
 
# # Upsample minority class
# dataset_minority_upsampled = resample(dataset_minority, 
#                                  replace = True,  # sample with replacement
#                                  n_samples = 144277,   # to match majority class
#                                  random_state = 123) # reproducible results
 
# # Combine majority class with upsampled minority class
# dataset_upsampled = pd.concat([dataset_majority, dataset_minority_upsampled])
 
# # Display new class counts
# dataset_upsampled.toxic.value_counts()

# Downsample majority class
dataset_majority_downsampled = resample(dataset_majority, replace = True, n_samples = len(dataset_minority),
                                        random_state = 123)

# Combine two classes
dataset_ds = pd.concat([dataset_minority, dataset_majority_downsampled])

# Shuffle dataset
dataset_ds = dataset_ds.sample(frac=1).reset_index(drop=True)

In [207]:
dataset_ds.shape

(30588, 8)

In [208]:
# preprocessing
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
vect_char = TfidfVectorizer(max_features=40000, lowercase=True, analyzer='char',
                        stop_words= 'english',ngram_range=(3,6),dtype=np.float32)

In [209]:
tr_vect.shape

(30588, 20000)

In [210]:
tr_vect = vect_word.fit_transform(dataset_ds['comment_text'])
ts_vect = vect_word.transform(test['comment_text'])

In [211]:
# print('before', tr_vect)
# tr_vect = normalize(tr_vect, norm = 'l2', axis = 0)
# print('after', tr_vect1)

In [212]:
X = sparse.hstack([tr_vect])
x_test = sparse.hstack([ts_vect])

In [213]:
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = dataset_ds[target_col]
print('X shape', X.shape)
print('y shape', y.shape)

X shape (30588, 20000)
y shape (30588, 6)


In [214]:
# train splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('x_train = ', X_train.shape)
print('X_test = ', X_test.shape)
print('y_train=', y_train.shape)
print('y_test', y_test.shape)

x_train =  (20493, 20000)
X_test =  (10095, 20000)
y_train= (20493, 6)
y_test (10095, 6)


In [215]:
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = dataset_ds[target_col][0:20493]
prd = np.zeros((10095, 6))
print(y.shape)

(20493, 6)


In [201]:
# regression part
prd = np.zeros((y_test.shape[0],y.shape[1]))
for i,col in enumerate(target_col):
    lr = LogisticRegression(C=2,random_state = i,class_weight = 'balanced')
    print('Building {} model for column:{''}'.format(i,col)) 
    lr.fit(X_train,y[col])
    prd[:,i] = lr.predict_proba(X_test)[:,1]\
    print(metrics.roc_auc_score(y_test.iloc[:, i] , pd.DataFrame(prd[:, i])))

Building 0 model for column:toxic
0.487611887154
Building 1 model for column:severe_toxic
0.555622568765
Building 2 model for column:obscene
0.4893497016
Building 3 model for column:threat
0.565021569548
Building 4 model for column:insult
0.504110198575
Building 5 model for column:identity_hate
0.572559558771
